In [1]:
import re #regex for password criteria checking
import getpass #password input box
import json
from pathlib import Path
import os.path
import os
import hashlib
from Crypto.Cipher import AES
import sys
import base64

In [2]:
###########################################################
# Functions 
# This function takes in an email and password, and returns true/false based on successful authentication
def loginAuthentication(usersObj, emailAddress, inputPassword):
    passwordHash = ' '
    if(usersObj.isUserRegistered(emailAddress)):
        passwordHash = hashPassword(inputPassword, usersObj.getPasswordHash(emailAddress)[:32])
    else:
        print('user does not exist')
        return False
    #print('password hash')
    #print(usersObj.getPasswordHash(emailAddress)[32:])
    #print('password input hash')
    #print(passwordHash[32:])
    #print('is user registered')
    #print(usersObj.isUserRegistered(emailAddress))
    #print('do hashes equal?')
    #print(usersObj.getPasswordHash(emailAddress)[32:] in passwordHash)
    if(usersObj.isUserRegistered(emailAddress) and usersObj.getPasswordHash(emailAddress)[32:] in passwordHash):
        return True #exits while loop
    else:
        print('Incorrect email/password combination\n')
        return False #keeps running loop until true
        print('\n')

# Takes a password and returns a sha-256 hashvalue with a salt
def hashPassword(password, salt) :
    # SHA hashing
    hashResult = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 200000) #200,000 iterations for SHA algorithm
    # Storing the hash and salt
    storage = salt + hashResult #[:32] is salt [32:] is key

    return salt + hashResult

def encryptContact(plaintext):
    session_key = get_random_bytes(16)
    aes_cipher = AES.new(session_key, AES.MODE_EAX)
    ciphertext,tag = aes_cipher.encrypt_and_digest(plaintext)
    
    return [session_key, ciphertext, tag, aes_cipher.nonce]

def decryptContact(decryptArray, session_key):
    nonce = decryptArray[2]
    tag = decryptArray[1]
    ciphertext = decryptArray[0]
    
    aes_cipher = AES.new(session_key, AES.MODE_EAX, nonce)
    plaintext = aes-cipher.decrypt_and_verify(ciphertext, tag)
    
    return plaintext

# New user generation, should generate a user and interface the file to save
def newUser():
    try_flag = True # used for password match checking

    fullName = input("Enter Full Name: ")
    email = input("Enter Email Address: ")
    while 1: # prompt the user until the passwords match
        if try_flag == False:
            # create junk values, user won't be created
            fullName = ""
            email = ""
            passWd = ""
            break
        passWd = input("Input Password: ")
        passWd_Check = input("Re-Enter Password: ")
        
        if passWd != passWd_Check:
            while 1: # prompt until accepted "try again?" answer
                try_again = input("Passwords do not match, try again? (y/n):")
                if (try_again == 'y' or try_again == 'Y'):
                    try_flag = True
                    break
                elif (try_again == 'n' or try_again == 'N'):
                    try_flag = False
                    break
                else: # invalid answer
                    print("User did not input y/n")
                
        else:
           # print("Passwords Match.")
            # Hashes the password using SHA-256
            # Use passWdHashed[32:] to get the key
            # Use passWdHashed[:32] to get the salt
            # 32 byte salt
            salt = os.urandom(32)
            passWdHashed = hashPassword(passWd, salt)
            break
    user = {
        "Name": fullName,
        "Email": email,
        "Password": passWdHashed
    }
    return user # return the user object to main to be saved

In [8]:
# data object that stores the user's 
class UserData:
    userData_file_path = Path(".SFTP_USERS.dat")
    temp_data_file_path = Path(".SFTP_USERS.tmp")
    data = {}
    enc_data = {}
    data_fp = None
    temp_fp = None
    
    def __init__(self,filePath=userData_file_path,tempFile=temp_data_file_path):
        if os.path.exists(filePath):
            self.data_fp = open(filePath, "w+")
            self.temp_fp = open(tempFile, "w+")
            try:
                self.data = json.load(self.data_fp)
            except:
                print('failed to load JSON file, is it empty?')
        else:
            self.data = dict()
            self.data_fp = open(filePath, "w+")
            self.temp_fp = open(tempFile, "w+")
        return None
    
    def __del__(self):
        self.writeData()
        return None
    
    def isEmpty(self):
        return not(bool(self.data))
    
    def addUser(self, user):
        self.data[user['Email']] = user
        self.data[user['Email']]['Password'] = base64.b64encode(user['Password']).decode('utf-8')
        self.data[user['Email']]['Contacts'] = {}
        #self.enc_data[user['Email']] = self.data[user['Email']]
    
    def isUserRegistered(self, email):
        if email in self.data.keys():
            return True
        else:
            return False
        
    def getPasswordHash(self, email):
        return base64.b64decode(self.data[email]["Password"].encode('utf-8'))
    
    def getContacts(self, email):
        if self.isUserRegistered(user):
            #self.decrpytContacts(email, self.getPasswordHash(email))
            return self.data[email]['Contacts']
        else:
            return []
    
    def doesContactExist(self, email, contact):
        if self.isUserRegistered(email) and self.data[email]['Contacts']['Email'] == contact['Email'] and self.data[email]['Contacts']['Name'] == contact['Name']:
            return True
        else:
            return False
    
    def addContact(self, email, contact):
        if (self.isUserRegistered(email)):
            self.data[email]['Contacts'] = contact
            #self.encryptContacts(email)
        else:
            print('user with specified email does not exist')
        return None
    
    def removeContact(self, email, contact):
        if self.doesContactExist(email, contact):
            self.data[email]['Contacts'].remove(contact)
            #self.encryptContacts(email)
            return True
        else:
            return False
    
    def loadData(self):
        # TODO:
        return None
    
    def writeData(self):
        try:
            json.dump(self.data, self.temp_fp)
            self.temp_fp.close()
            self.data_fp.close()
            os.remove(self.userData_file_path)
            os.rename(self.temp_data_file_path, self.userData_file_path)
        except OSError:
            return False
        return True
    
    def decryptContacts(self, email, passwordHash):
        if self.data[email]['Password'] == passwordHash:
            self.data[email] = decryptContact(passwordHash, json.loads(self.enc_data[email]['Contacts']))
            return True
        else:
            return False
        
    def encryptContacts(self, email):
        self.enc_data[email]['Contacts'] = encryptContact(json.dumps(self.data[email]['Contacts']).encode('utf-8'), self.data[email]['Password'])
        return None







In [9]:
###########################################################
# Start of main program
#main MD5
md5Checksum = " "
#sets initial MD5 checksum to compare to
if os.path.exists(Path(".SFTP_USERS.dat")):
    with open(".SFTP_USERS.dat","rb+") as f:
        data = f.read()
        print(data)
        md5Checksum = hashlib.md5(data).hexdigest()

usersObj = UserData() #main user data in memory
userList = not usersObj.isEmpty() # If there are no users in file 
#samuel insert code around here for file checking

#If there are no users in file, create initial user in while loop
while (userList == False):
    print('No users are registered with this client')
    firstUser= input('Would you like to register a new user now? (y/n): ')
    
    # Check checksum for changes
    with open(".SFTP_USERS.dat","rb") as f:
        data = f.read()
        tempSum = hashlib.md5(data).hexdigest()
        if (md5Checksum != tempSum):
            print('hacked .SFTP_USERS.dat detected')
            sys.exit()
                    
    if (firstUser == 'Y' or firstUser == 'y'):
        
        # create the new user
        user = newUser()
        if (user["Name"] == "" or user["Email"] == "" or user["Password"] == ""): # was a contact unable to be created
            print('You cannot use this system without a user registered')
        else:
            
            # Check checksum for changes
            with open(".SFTP_USERS.dat","rb") as f:
                data = f.read()
                tempSum = hashlib.md5(data).hexdigest()
                if (md5Checksum != tempSum):
                    print('hacked .SFTP_USERS.dat detected')
                    sys.exit()
                    
            # Create new user
            usersObj.addUser(user)
            
            # Update checksum
            with open(".SFTP_USERS.dat","rb") as f:
                data = f.read()
                md5Checksum = hashlib.md5(data).hexdigest()
                 
        userList = True #can be changed with user registration code (must not == False to exit loop)
    else:
        print('You cannot use this system without a user registered')

print('\n$Welcome to Secure Drop!$ \n--- Login ---')

loginSuccess = False # Exits the while loop when not False
while (loginSuccess == False):
    emailAddress = input('Email Address:')
    inputPassword = input('Password:')
    
    # Check checksum for changes
    with open(".SFTP_USERS.dat","rb") as f:
        data = f.read()
        tempSum = hashlib.md5(data).hexdigest()
        if (md5Checksum != tempSum):
            print('hacked .SFTP_USERS.dat detected')
            sys.exit()
            
     # Update the success variable       
    loginSuccess = loginAuthentication(usersObj, emailAddress, inputPassword) # Changes the boolean to True if successful
print("Login was successful!")

exit_shell = False # Exits the while loop when not False
while (exit_shell == False):
    # 'shell' prefix for each line
    textInput = input('$secure_drop>')
    
    # Check checksum for changes
    with open(".SFTP_USERS.dat","rb") as f:
        data = f.read()
        tempSum = hashlib.md5(data).hexdigest()
        if (md5Checksum != tempSum):
            print('hacked .SFTP_USERS.dat detected')
            sys.exit()
    
    # Add command
    if(textInput == 'add'):
        # Check to verify user is correct
        print('Verify your identity: ')
        verifyLoginSuccess = False
        while(verifyLoginSuccess is False):
            checkEmail = input('Your Email address: ')
            checkPassword = input('Your Password: ')
            verifyLoginSuccess = loginAuthentication(usersObj,checkEmail,checkPassword)
        
        # Check checksum for changes
        with open(".SFTP_USERS.dat","rb") as f:
            data = f.read()
            tempSum = hashlib.md5(data).hexdigest()
            if (md5Checksum != tempSum):
                print('hacked .SFTP_USERS.dat detected')
                sys.exit()
                
        # Adds users to the contact list for the logged in user
        print('\n--- Add Contact ---')
        contact_name = input('Enter Full Name: ')
        contact_email = input('Enter Email Address: ')

        encrypted_email = contact_email
        # create the contact dict
        contact = {
            "Email": encrypted_email,
            "Name": contact_name
        }
        # add contact to user 
        usersObj.addContact(emailAddress, contact)
        
        # Update checksum
        with open(".SFTP_USERS.dat","rb") as f:
            data = f.read()
            md5Checksum = hashlib.md5(data).hexdigest()
        
        # was the contact properly stored?
        if(usersObj.doesContactExist(emailAddress, contact)): #needs to check our implementation of user storage
            print('Contact added to List')
        else:
            print('Failed to add contact')
            
    # List command
    elif(textInput == 'list'):
        # Check to verify user is correct
        print('Verify your identity: ')
        verifyLoginSuccess = False
        while(verifyLoginSuccess is False):
            checkEmail = input('Your Email address: ')
            checkPassword = input('Your Password: ')
            verifyLoginSuccess = loginAuthentication(usersObj,checkEmail,checkPassword)
               
        
    # Send command
    elif(textInput == 'send'):
        print('test2')
        
    
    # Help command
    elif(textInput == 'help'):
        
        # Check checksum for changes
        with open(".SFTP_USERS.dat","rb") as f:
            data = f.read()
            tempSum = hashlib.md5(data).hexdigest()
            if (md5Checksum != tempSum):
                print('hacked .SFTP_USERS.dat detected')
                sys.exit()
                
        print('\t"add" -> Add a new contact \n\t\"list\" -> List all online contacts \n\t\"send\" -> Transfer file to contact \n\t\"exit\" -> Exit SecureDrop')
        
    # Exit command
    elif(textInput == 'exit'):
        exit_shell= True
        
print('Exiting Secure Drop')

b''
failed to load JSON file, is it empty?
No users are registered with this client
Would you like to register a new user now? (y/n): y
Enter Full Name: k
Enter Email Address: kl
Input Password: ff
Re-Enter Password: ff

$Welcome to Secure Drop!$ 
--- Login ---
Email Address:kl
Password:ff
Login was successful!
$secure_drop>add
Verify your identity: 
Your Email address: kl
Your Password: ff

--- Add Contact ---
Enter Full Name: t
Enter Email Address: tl
{'Email': 'tl', 'Name': 't'}
kl
{'Email': 'tl', 'Name': 't'}
True
{'kl': {'Name': 'k', 'Email': 'kl', 'Password': 'AaFA5trtF8F0bGIRfkfXPSzfd+MkzuuZOC5r9uZXkLx966A6T5Wucv1gpSsc31mAaNAP7CqntzWE4/ULEUdTaw==', 'Contacts': {'Email': 'tl', 'Name': 't'}}}
{'Name': 'k', 'Email': 'kl', 'Password': 'AaFA5trtF8F0bGIRfkfXPSzfd+MkzuuZOC5r9uZXkLx966A6T5Wucv1gpSsc31mAaNAP7CqntzWE4/ULEUdTaw==', 'Contacts': {'Email': 'tl', 'Name': 't'}}
{'Email': 'tl', 'Name': 't'}
tl
tl
t
t
Contact added to List
$secure_drop>exit
Exiting Secure Drop


In [10]:
usersObj.__del__()

{"kl": {"Name": "k", "Email": "kl", "Password": "AaFA5trtF8F0bGIRfkfXPSzfd+MkzuuZOC5r9uZXkLx966A6T5Wucv1gpSsc31mAaNAP7CqntzWE4/ULEUdTaw==", "Contacts": {"Email": "tl", "Name": "t"}}}


In [ ]:
b'\\\x11\xb2\xb5\xef\x81\x1aI\xbe.\xd4I\xfb\xf1\x07^\xac\x90#c\x9a\x18\xf8v\x8e\xdc\x01\xe6\xad\x91\x19c\x9f\x9a\x11p\xd8\x1c\x99\xc4\xdd\x07\xd2d\xb9\x05\xfc7\xfeP\xf6)\xd5^\x06\\\x15\x03\x8a\x96\x98\x88m\xab'.encode('base64')